## GIS-test

Eit forsøk på å gjera litt meir ut av datasettet frå [dette innlegget](./posts/230425_jordskjelv/index.ipynb)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

Eg tek her utgangspunkt i `geodataframe`-en me laga i jordskjelvprosjektet:

In [2]:
#| echo: false
skjelv = pd.read_csv("skjelv.csv")
skjelv = skjelv[skjelv["type"] == "earthquake"]
skjelv = skjelv[["time", "latitude", "longitude", "depth", "mag"]]

skjelv_gdf = gpd.GeoDataFrame(
    skjelv, 
    geometry = gpd.points_from_xy(
        skjelv.longitude, 
        skjelv.latitude
        )
    )
skjelv_gdf.head()


,time,latitude,longitude,depth,mag,geometry
0,2023-04-24T20:00:55.086Z,-0.7810,98.5339,15.462,7.1,POINT (98.53390 -0.78100)
1,2023-04-24T02:17:56.596Z,-60.3893,-26.1923,10.000,5.6,POINT (-26.19230 -60.38930)
2,2023-04-24T00:41:55.460Z,-29.9676,-177.8264,43.063,7.1,POINT (-177.82640 -29.96760)
3,2023-04-22T21:17:45.986Z,-0.9589,98.4839,10.000,5.7,POINT (98.48390 -0.95890)
4,2023-04-22T17:09:45.448Z,-0.7645,98.5780,27.385,5.8,POINT (98.57800 -0.76450)


I tillegg bruker me datasettet som me laga bakgrunnskart frå: 

In [9]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


Me kan prøva å finna ut kor mange gongar kvart land er råka av jordskjelv i tabellen, og sortera dei ti med flest "treff". 

In [13]:
# lager ein ny gdf med land som indeks
land = world.copy()
land.set_index('name', inplace=True)
land.head()

,pop_est,continent,iso_a3,gdp_md_est,geometry
name,,,,,
Fiji,889953.0,Oceania,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
Tanzania,58005463.0,Africa,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
W. Sahara,603253.0,Africa,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
Canada,37589262.0,North America,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
United States of America,328239523.0,North America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [16]:
land["n_skjelv"] = land.apply(lambda x:
     skjelv_gdf['geometry'].within(x['geometry']).sum(), axis=1)

land.head()

,pop_est,continent,iso_a3,gdp_md_est,geometry,n_skjelv
name,,,,,,
Fiji,889953.0,Oceania,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0
Tanzania,58005463.0,Africa,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",17
W. Sahara,603253.0,Africa,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0
Canada,37589262.0,North America,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",19
United States of America,328239523.0,North America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",149


In [32]:
land.sort_values(by = ["n_skjelv"], ascending=False).head(10)

,pop_est,continent,iso_a3,gdp_md_est,geometry,n_skjelv,gdp_pc
name,,,,,,,
Indonesia,2.706256e+08,Asia,IDN,1119190,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ...",404,0.004136
Papua New Guinea,8.776109e+06,Oceania,PNG,24829,"MULTIPOLYGON (((141.00021 -2.60015, 142.73525 ...",392,0.002829
Chile,1.895204e+07,South America,CHL,282318,"MULTIPOLYGON (((-68.63401 -52.63637, -68.63335...",329,0.014896
China,1.397715e+09,Asia,CHN,14342903,"MULTIPOLYGON (((109.47521 18.19770, 108.65521 ...",258,0.010262
Peru,3.251045e+07,South America,PER,226848,"POLYGON ((-69.89364 -4.29819, -70.79477 -4.251...",205,0.006978
Argentina,4.493871e+07,South America,ARG,445445,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000...",182,0.009912
Mexico,1.275755e+08,North America,MEX,1268870,"POLYGON ((-117.12776 32.53534, -115.99135 32.6...",157,0.009946
Philippines,1.081166e+08,Asia,PHL,376795,"MULTIPOLYGON (((120.83390 12.70450, 120.32344 ...",156,0.003485
Japan,1.262649e+08,Asia,JPN,5081769,"MULTIPOLYGON (((141.88460 39.18086, 140.95949 ...",156,0.040247


In [25]:
land["gdp_pc"] = land["gdp_md_est"] / land["pop_est"]
land.head()

,pop_est,continent,iso_a3,gdp_md_est,geometry,n_skjelv,gdp_pc
name,,,,,,,
Fiji,889953.0,Oceania,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0,0.006176
Tanzania,58005463.0,Africa,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",17,0.001089
W. Sahara,603253.0,Africa,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0,0.001504
Canada,37589262.0,North America,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",19,0.046195
United States of America,328239523.0,North America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",149,0.065298
